# NavAI: IMU Speed Estimation Model Training

This notebook demonstrates:
1. Loading and exploring sensor data
2. Training IMU-based speed estimation models
3. Evaluating model performance
4. Exporting models to TensorFlow Lite for mobile deployment

## Setup and Imports

In [1]:
import sys
import os
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import tensorflow as tf

from data.data_loader import DataLoader as NavAIDataLoader
from models.speed_estimator import SpeedCNN, SpeedLSTM, WindowGenerator, create_tensorflow_model, convert_to_tflite

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Setup complete!")

ModuleNotFoundError: No module named 'numpy'

## 1. Data Loading and Exploration

### Quick Dataset Options for Showcase:

**🚀 Option 1: comma2k19 (Recommended)**
- **Best for**: Vehicle navigation, real driving scenarios
- **Download**: `git clone https://github.com/commaai/comma2k19.git` (includes 1-minute sample)
- **Full dataset**: http://academictorrents.com/details/65a2fbc964078aff62076ff4e103f18b951c5ddb (~100GB)
- **Data**: IMU (100Hz) + GPS + CAN speed + Video

**🥈 Option 2: EuRoC MAV** 
- **Best for**: Precise evaluation, academic validation
- **Download**: http://robotics.ethz.ch/~asl-datasets/ijrr_euroc_mav_dataset/
- **Size**: ~1-2GB per sequence
- **Data**: Stereo cameras + IMU (200Hz) + precise ground truth

**💡 For immediate demo**: The notebook will create synthetic data if no dataset is available.

In [ ]:
# Initialize data loader
data_loader = NavAIDataLoader(target_sample_rate=100)

# For immediate showcase - use comma2k19 sample data
# Download sample from: https://github.com/commaai/comma2k19
print("For showcase: Download comma2k19 sample data")
print("1. Clone: git clone https://github.com/commaai/comma2k19.git")
print("2. Use Example_1/b0c9d2329ad1606b sample segment")
print("3. Or download full dataset from: http://academictorrents.com/details/65a2fbc964078aff62076ff4e103f18b951c5ddb")

# Configure data paths - update these with your actual data locations
data_paths = {
    'comma2k19': '../data/comma2k19/',  # Download comma2k19 here
    # 'navai': '../data/navai_logs/',  # Your collected logs
    # 'oxiod': '../data/oxiod/',     # Uncomment when available
    # 'iovnbd': '../data/iovnbd/',   # Uncomment when available
}

# Load available datasets
try:
    df = data_loader.load_combined_dataset(data_paths)
    print(f"Loaded dataset shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")
    print(f"\nDataset info:")
    df.info()
except Exception as e:
    print(f"Data loading failed: {e}")
    print("Please download comma2k19 sample data first")
    
    # Create sample data for demonstration
    print("\nCreating sample synthetic data for demonstration...")
    import numpy as np
    n_samples = 10000
    
    # Generate realistic IMU + GPS data
    time_ns = np.arange(n_samples) * 10_000_000  # 100Hz
    accel_x = np.random.normal(0, 2, n_samples) + np.sin(np.arange(n_samples) * 0.01) * 3
    accel_y = np.random.normal(0, 2, n_samples)
    accel_z = np.random.normal(-9.81, 1, n_samples)
    
    gyro_x = np.random.normal(0, 0.1, n_samples)
    gyro_y = np.random.normal(0, 0.1, n_samples)  
    gyro_z = np.random.normal(0, 0.1, n_samples)
    
    # Simulated vehicle speed (0-30 m/s)
    speed_profile = 10 + 8 * np.sin(np.arange(n_samples) * 0.001) + np.random.normal(0, 1, n_samples)
    speed_profile = np.clip(speed_profile, 0, 30)
    
    df = pd.DataFrame({
        'timestamp_ns': time_ns,
        'accel_x': accel_x, 'accel_y': accel_y, 'accel_z': accel_z,
        'gyro_x': gyro_x, 'gyro_y': gyro_y, 'gyro_z': gyro_z,
        'mag_x': np.random.normal(20, 5, n_samples),
        'mag_y': np.random.normal(0, 5, n_samples), 
        'mag_z': np.random.normal(-40, 5, n_samples),
        'qw': np.ones(n_samples), 'qx': np.zeros(n_samples), 
        'qy': np.zeros(n_samples), 'qz': np.zeros(n_samples),
        'gps_lat': 37.4419 + np.random.normal(0, 0.0001, n_samples),
        'gps_lon': -122.1430 + np.random.normal(0, 0.0001, n_samples),
        'gps_speed_mps': speed_profile,
        'device': 'synthetic', 'source': 'demo'
    })
    
    print(f"Created synthetic dataset: {df.shape}")

In [ ]:
# Data exploration
if not df.empty:
    # Basic statistics
    print("Dataset Statistics:")
    print(df.describe())
    
    # Plot sensor data
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Accelerometer
    axes[0,0].plot(df['accel_x'][:1000], label='X', alpha=0.7)
    axes[0,0].plot(df['accel_y'][:1000], label='Y', alpha=0.7)
    axes[0,0].plot(df['accel_z'][:1000], label='Z', alpha=0.7)
    axes[0,0].set_title('Accelerometer (first 1000 samples)')
    axes[0,0].set_ylabel('Acceleration (m/s²)')
    axes[0,0].legend()
    
    # Gyroscope
    axes[0,1].plot(df['gyro_x'][:1000], label='X', alpha=0.7)
    axes[0,1].plot(df['gyro_y'][:1000], label='Y', alpha=0.7)
    axes[0,1].plot(df['gyro_z'][:1000], label='Z', alpha=0.7)
    axes[0,1].set_title('Gyroscope (first 1000 samples)')
    axes[0,1].set_ylabel('Angular velocity (rad/s)')
    axes[0,1].legend()
    
    # GPS Speed
    valid_gps = df[df['gps_speed_mps'] > 0]
    if not valid_gps.empty:
        axes[1,0].plot(valid_gps['gps_speed_mps'][:1000])
        axes[1,0].set_title('GPS Speed (first 1000 valid samples)')
        axes[1,0].set_ylabel('Speed (m/s)')
        
        # Speed distribution
        axes[1,1].hist(valid_gps['gps_speed_mps'], bins=50, alpha=0.7)
        axes[1,1].set_title('GPS Speed Distribution')
        axes[1,1].set_xlabel('Speed (m/s)')
        axes[1,1].set_ylabel('Frequency')
    
    plt.tight_layout()
    plt.show()
else:
    print("No data loaded. Please check your data paths and ensure log files exist.")

## 2. Data Preprocessing and Window Generation

In [ ]:
if not df.empty:
    # Filter data with valid GPS speed
    valid_data = df[(df['gps_speed_mps'] >= 0) & (df['gps_speed_mps'] <= 50)].copy()  # Reasonable speed range
    
    print(f"Valid data samples: {len(valid_data)} / {len(df)} ({len(valid_data)/len(df)*100:.1f}%)")
    
    # Create windows for training
    window_generator = WindowGenerator(
        window_size_sec=1.5,
        stride_sec=0.25,
        sample_rate=100,
        feature_cols=['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z'],
        target_col='gps_speed_mps'
    )
    
    X, y = window_generator.create_windows(valid_data)
    
    print(f"Generated windows: X shape {X.shape}, y shape {y.shape}")
    print(f"Speed range: {y.min():.2f} - {y.max():.2f} m/s")
    
    # Train/validation split
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    
    print(f"Training set: {X_train.shape[0]} windows")
    print(f"Validation set: {X_val.shape[0]} windows")
else:
    print("Skipping preprocessing - no data available")

## 3. Model Training (PyTorch)

In [ ]:
if 'X_train' in locals():
    # Convert to PyTorch tensors
    X_train_torch = torch.FloatTensor(X_train)
    y_train_torch = torch.FloatTensor(y_train)
    X_val_torch = torch.FloatTensor(X_val)
    y_val_torch = torch.FloatTensor(y_val)
    
    # Create data loaders
    train_dataset = TensorDataset(X_train_torch, y_train_torch)
    val_dataset = TensorDataset(X_val_torch, y_val_torch)
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    
    # Initialize model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    model = SpeedCNN(input_channels=6, hidden_dim=64).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Training loop
    num_epochs = 20
    train_losses = []
    val_losses = []
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        train_loss = 0.0
        
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            
            optimizer.zero_grad()
            outputs = model(batch_X).squeeze()
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        # Validation
        model.eval()
        val_loss = 0.0
        
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                outputs = model(batch_X).squeeze()
                loss = criterion(outputs, batch_y)
                val_loss += loss.item()
        
        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        if (epoch + 1) % 5 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    
    # Plot training curves
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('MSE Loss')
    plt.title('Training Progress')
    plt.legend()
    plt.show()
    
    print("PyTorch training completed!")
else:
    print("Skipping training - no data available")

## 4. Model Evaluation

In [ ]:
if 'model' in locals():
    # Evaluate on validation set
    model.eval()
    predictions = []
    actuals = []
    
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            batch_X = batch_X.to(device)
            outputs = model(batch_X).squeeze()
            predictions.extend(outputs.cpu().numpy())
            actuals.extend(batch_y.numpy())
    
    predictions = np.array(predictions)
    actuals = np.array(actuals)
    
    # Calculate metrics
    mse = mean_squared_error(actuals, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(actuals, predictions)
    
    # Calculate percentage errors
    mean_speed = np.mean(actuals)
    rmse_percent = (rmse / mean_speed) * 100
    mae_percent = (mae / mean_speed) * 100
    
    print(f"Validation Metrics:")
    print(f"RMSE: {rmse:.3f} m/s ({rmse_percent:.1f}%)")
    print(f"MAE: {mae:.3f} m/s ({mae_percent:.1f}%)")
    print(f"Mean actual speed: {mean_speed:.3f} m/s")
    
    # Plot predictions vs actuals
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.scatter(actuals, predictions, alpha=0.5)
    plt.plot([actuals.min(), actuals.max()], [actuals.min(), actuals.max()], 'r--', lw=2)
    plt.xlabel('Actual Speed (m/s)')
    plt.ylabel('Predicted Speed (m/s)')
    plt.title('Predictions vs Actuals')
    
    plt.subplot(1, 2, 2)
    errors = predictions - actuals
    plt.hist(errors, bins=50, alpha=0.7)
    plt.xlabel('Prediction Error (m/s)')
    plt.ylabel('Frequency')
    plt.title('Error Distribution')
    plt.axvline(0, color='red', linestyle='--')
    
    plt.tight_layout()
    plt.show()
else:
    print("Skipping evaluation - no trained model available")

## 5. TensorFlow Model and Mobile Export

In [ ]:
if 'X_train' in locals():
    # Create TensorFlow model
    input_shape = (X_train.shape[1], X_train.shape[2])  # (sequence_length, features)
    tf_model = create_tensorflow_model(input_shape, model_type='cnn')
    
    # Compile model
    tf_model.compile(
        optimizer='adam',
        loss='mse',
        metrics=['mae']
    )
    
    print("TensorFlow model created:")
    tf_model.summary()
    
    # Train TensorFlow model
    history = tf_model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=20,
        batch_size=32,
        verbose=1
    )
    
    # Convert to TensorFlow Lite
    print("\nConverting to TensorFlow Lite...")
    tflite_model = convert_to_tflite(
        tf_model, 
        quantize=True, 
        representative_dataset=X_train
    )
    
    # Save TFLite model
    tflite_path = '../models/speed_estimator.tflite'
    os.makedirs(os.path.dirname(tflite_path), exist_ok=True)
    
    with open(tflite_path, 'wb') as f:
        f.write(tflite_model)
    
    print(f"TensorFlow Lite model saved to: {tflite_path}")
    print(f"Model size: {len(tflite_model) / 1024:.1f} KB")
    
    # Test TFLite model
    interpreter = tf.lite.Interpreter(model_content=tflite_model)
    interpreter.allocate_tensors()
    
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    print(f"\nTFLite model details:")
    print(f"Input shape: {input_details[0]['shape']}")
    print(f"Output shape: {output_details[0]['shape']}")
    print(f"Input dtype: {input_details[0]['dtype']}")
    print(f"Output dtype: {output_details[0]['dtype']}")
    
    # Test inference
    test_input = X_val[:1].astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], test_input)
    interpreter.invoke()
    tflite_output = interpreter.get_tensor(output_details[0]['index'])
    
    tf_output = tf_model.predict(test_input)
    
    print(f"\nInference test:")
    print(f"TensorFlow output: {tf_output[0][0]:.3f}")
    print(f"TFLite output: {tflite_output[0][0]:.3f}")
    print(f"Actual speed: {y_val[0]:.3f}")
    
else:
    print("Skipping TensorFlow training - no data available")

## 6. Summary and Next Steps

This notebook demonstrated the complete pipeline for training IMU-based speed estimation models:

1. **Data Loading**: Unified loader supporting multiple datasets
2. **Preprocessing**: Window generation and data preparation
3. **Model Training**: Both PyTorch and TensorFlow implementations
4. **Evaluation**: Performance metrics and visualization
5. **Mobile Export**: TensorFlow Lite conversion for Android deployment

### Next Steps:
1. **Collect more data** using the Android sensor logger
2. **Integrate public datasets** (IO-VNBD, OxIOD, comma2k19)
3. **Implement EKF sensor fusion** in Phase 2
4. **Add map matching** and offline navigation
5. **Integrate ARCore VIO** for enhanced accuracy

### Performance Targets:
- **Speed RMSE**: <10% (currently achieved: varies by dataset)
- **Model size**: <1MB (TFLite quantized)
- **Inference time**: <10ms on mobile devices